In [6]:
from sklearn.datasets import load_files
from keras.utils import np_utils
import numpy as np
import pandas as pd
from glob import glob
from IPython.core.display import Image, display
import cv2                
import matplotlib.pyplot as plt                        
%matplotlib inline

from sklearn.metrics import classification_report

from keras import models, layers, optimizers
from keras.preprocessing import image
from keras.applications.vgg19 import VGG19
from keras.applications.resnet50 import ResNet50
from keras.applications.resnet50 import preprocess_input, decode_predictions
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import VGG16, DenseNet121
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.callbacks import ModelCheckpoint
from tqdm import tqdm
import math
import pickle
import os
from PIL import ImageFile
import time

from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.layers import Dropout, Flatten, Dense, Activation
from keras.models import Sequential
import tensorflow as tf
from tensorflow.keras.callbacks import TensorBoard
# from keras.callbacks import TensorBoard

In [2]:
# define function to load train, test, and validation datasets
def load_dataset(path):
    """Returns the path and the Label from the folder"""
    data = load_files(path)
    chest_files = np.array(data['filenames'])
    chest_targets = np_utils.to_categorical(np.array(data['target']), 2)
    return chest_files, chest_targets

# load train, test, and validation datasets
train_files, train_targets = load_dataset('../imgs/images/train')
test_files, test_targets = load_dataset('../imgs/images/test')

# load list of dog names
labels = [item[21:-1] for item in sorted(glob("../imgs/images/train/*/"))]
CLASSES = len(labels)

print('Statistics about the Dataset:\n')
print('There are %d total chest deseases.' % len(labels))
print('There are %s total chest images.\n' % len(np.hstack([train_files, test_files])))
print('There are %d training chest images.' % len(train_files))
print('There are %d test chest images.'% len(test_files))

Statistics about the Dataset:

There are 2 total chest deseases.
There are 4999 total chest images.

There are 4032 training chest images.
There are 967 test chest images.


# Create Generators

In [9]:
img_width, img_height, channels = 365, 365, 3

top_model_weights_path = 'bottleneck_fc_model.h5'
train_data_dir = '../imgs/images/train'
test_data_dir = '../imgs/images/test'


train_batch_size = 20
test_batch_size = 10


print('**********************TRAIN GENERATOR**********************')
### Train Generator
train_datagen = ImageDataGenerator()


train_generator = train_datagen.flow_from_directory(train_data_dir,
                                                    target_size=(img_width, img_height),
                                                    batch_size = train_batch_size,
                                                    class_mode = 'categorical',
                                                    shuffle=False)
                                                    #color_mode = 'grayscale'
    

print('**********************TEST GENERATOR**********************')
### Test Generator
test_datagen = ImageDataGenerator()
test_generator = test_datagen.flow_from_directory(test_data_dir,
                                                    target_size=(img_width, img_height),
                                                    batch_size=test_batch_size,
                                                    class_mode='categorical',
                                                    shuffle=False)

**********************TRAIN GENERATOR**********************
Found 4032 images belonging to 2 classes.
**********************TEST GENERATOR**********************
Found 967 images belonging to 2 classes.


# CNN
## Architecture

In [11]:
nb_train_samples = 4032
nb_test_samples = 967

# Define the architecture
model = DenseNet121(weights= 'imagenet', include_top=False, input_shape=(img_height, img_width, channels)) # or weights=None

# Freeze some layers
for layer in model.layers[:]:
    layer.trainable = False

Chest_Dens_121_model = Sequential()
Chest_Dens_121_model.add(model)

Chest_Dens_121_model.add(layers.Flatten())

Chest_Dens_121_model.add(layers.Dense(256))
Chest_Dens_121_model.add(layers.BatchNormalization())
Chest_Dens_121_model.add(layers.Activation('relu'))
Chest_Dens_121_model.add(layers.Dropout(0.5))

Chest_Dens_121_model.add(layers.Dense(2))
Chest_Dens_121_model.add(layers.BatchNormalization())
Chest_Dens_121_model.add(layers.Activation('softmax'))
Chest_Dens_121_model.add(layers.Dropout(0.5))

adam = optimizers.Adam(lr=0.0001)
Chest_Dens_121_model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])

Chest_Dens_121_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
densenet121 (Model)          (None, 11, 11, 1024)      7037504   
_________________________________________________________________
flatten_4 (Flatten)          (None, 123904)            0         
_________________________________________________________________
dense_7 (Dense)              (None, 256)               31719680  
_________________________________________________________________
batch_normalization_7 (Batch (None, 256)               1024      
_________________________________________________________________
activation_56 (Activation)   (None, 256)               0         
_________________________________________________________________
dropout_7 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_8 (Dense)              (None, 2)                 514       
__________

## Fit

In [13]:
checkpointer = ModelCheckpoint(filepath='weights.best.RESN50.hdf5', verbose=1, save_best_only=True)

History = Chest_Dens_121_model.fit_generator(train_generator, 
                                        epochs=1,
                                        validation_data = test_generator,
                                        validation_steps = test_generator.samples / test_generator.batch_size,  
                                        steps_per_epoch = train_generator.samples / train_generator.batch_size, 
                                        callbacks=[checkpointer], verbose=1, shuffle=False)
#use_multiprocessing=True

Epoch 1/1


197/201 [============================>.] - ETA: 2:50:57 - loss: 6.1991 - acc: 0.45 - ETA: 2:50:52 - loss: 5.6156 - acc: 0.51 - ETA: 3:05:21 - loss: 5.6941 - acc: 0.49 - ETA: 3:10:51 - loss: 6.2298 - acc: 0.45 - ETA: 3:13:46 - loss: 5.9918 - acc: 0.46 - ETA: 3:16:01 - loss: 6.0490 - acc: 0.46 - ETA: 3:17:26 - loss: 5.9537 - acc: 0.46 - ETA: 3:18:36 - loss: 5.7955 - acc: 0.47 - ETA: 3:19:13 - loss: 5.9048 - acc: 0.47 - ETA: 3:19:33 - loss: 6.0905 - acc: 0.47 - ETA: 3:19:40 - loss: 6.0628 - acc: 0.47 - ETA: 3:19:04 - loss: 6.0776 - acc: 0.47 - ETA: 3:18:07 - loss: 6.0864 - acc: 0.47 - ETA: 3:17:10 - loss: 6.1908 - acc: 0.46 - ETA: 3:16:17 - loss: 6.2453 - acc: 0.46 - ETA: 3:15:24 - loss: 6.2931 - acc: 0.45 - ETA: 3:15:10 - loss: 6.3367 - acc: 0.45 - ETA: 3:14:35 - loss: 6.3558 - acc: 0.45 - ETA: 3:14:00 - loss: 6.3710 - acc: 0.45 - ETA: 3:13:16 - loss: 6.3217 - acc: 0.45 - ETA: 3:12:28 - loss: 6.3039 - acc: 0.45 - ETA: 3:11:21 - loss: 6.2156 - acc: 0.45 - ETA: 3:10:14 - loss: 6.1455 - acc

In [37]:
X = []
y = []

for features, label in train_generator:
    X.append(features)
    y.append(label)
X = np.array(X).reshape(-1, img_width, img_height, 1)

pickle_out = pickle.open('X_DENSNET.pickle', "wb")
pickle.dump(X, pickle_out)
pickle_out.close()

pickle_out = pickle.open('y_DENSNET.pickle', "wb")
pickle.dump(y, pickle_out)
pickle_out.close()

MemoryError: 

In [35]:
def path_to_tensor(img_path):
    # loads RGB image as PIL.Image.Image type
    img = image.load_img(img_path, target_size=(365, 365))
    # convert PIL.Image.Image type to 3D tensor with shape (224, 224, 3)
    x = image.img_to_array(img)
    # convert 3D tensor to 4D tensor with shape (1, 224, 224, 3) and return 4D tensor
    return np.expand_dims(x, axis=0)

def paths_to_tensor(img_paths):
    list_of_tensors = [path_to_tensor(img_path) for img_path in tqdm(img_paths)]
    return np.vstack(list_of_tensors)

ImageFile.LOAD_TRUNCATED_IMAGES = True
# pre-process the data for Keras
train_tensors = paths_to_tensor(train_files).astype('float32')/255
test_tensors = paths_to_tensor(test_files).astype('float32')/255



  0%|          | 0/4032 [00:00<?, ?it/s]

  0%|          | 5/4032 [00:00<01:42, 39.16it/s]

  0%|          | 10/4032 [00:00<01:37, 41.29it/s]

  0%|          | 15/4032 [00:00<01:34, 42.42it/s]

  0%|          | 20/4032 [00:00<01:31, 43.99it/s]

  1%|          | 25/4032 [00:00<01:30, 44.34it/s]

  1%|          | 30/4032 [00:00<01:30, 44.11it/s]

  1%|          | 35/4032 [00:00<01:28, 44.96it/s]

  1%|          | 40/4032 [00:00<01:31, 43.84it/s]

  1%|          | 45/4032 [00:01<01:28, 45.19it/s]

  1%|          | 50/4032 [00:01<01:26, 46.05it/s]

  1%|▏         | 55/4032 [00:01<01:25, 46.51it/s]

  1%|▏         | 60/4032 [00:01<01:24, 47.00it/s]

  2%|▏         | 65/4032 [00:01<01:32, 43.09it/s]

  2%|▏         | 70/4032 [00:01<01:36, 41.22it/s]

  2%|▏         | 75/4032 [00:01<01:34, 41.77it/s]

  2%|▏         | 80/4032 [00:01<01:32, 42.62it/s]

  2%|▏         | 85/4032 [00:01<01:37, 40.52it/s]

  2%|▏         | 90/4032 [00:02<01:42, 38.62it/s]

  2%|▏         | 95/4032 [00:02<01:39, 

MemoryError: 



  5%|▌         | 217/4032 [00:23<01:26, 44.35it/s]

In [30]:
test_tensors.shape, test_targets.shape

((967, 224, 224, 3), (967, 2))

In [31]:
### Load the model weights with the best validation loss.
Chest_Dens_121_model.load_weights('weights.best.RESN50.hdf5')

### Calculate classification accuracy on the test dataset.
# get index of predicted dog breed for each image in test set
predictions = [np.argmax(Chest_Dens_121_model.predict(np.expand_dims(feature, axis=0))) for feature in test_files]

# report test accuracy
test_accuracy = 100*np.sum(np.array(predictions)==np.argmax(test_targets, axis=1))/len(predictions)
print('Test accuracy: %.4f%%' % test_accuracy)

ValueError: Error when checking input: expected densenet121_input to have 4 dimensions, but got array with shape (1, 1)

In [ ]:
plt.figure(1, figsize=(10,10))  

# summarize history for accuracy  

plt.subplot(211)  
plt.plot(History.history['acc'])  
plt.plot(History.history['val_acc'])  
plt.title('Model Accuracy')  
plt.ylabel('Accuracy')  
plt.xlabel('Epoch')  
plt.legend(['train', 'val'], loc='upper left')  

# summarize history for loss  

plt.subplot(212)  
plt.plot(History.history['loss'])  
plt.plot(History.history['val_loss'])  
plt.title('Model Loss')  
plt.ylabel('Loss')  
plt.xlabel('Epoch')  
plt.legend(['train', 'val'], loc='upper left')  
plt.show()